In [35]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import io
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


In [36]:
SOS_token = 0
EOS_token = 1
UNK_token = 2

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS",  2:"UNK"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [37]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def Reverse(lst): 
    return [ele for ele in reversed(lst)] 


In [62]:
def readLangs(lang1, lang2, reverse=False, reverse_lang=False):
    print("Reading lines...")

    # Read the file and split into lines  
    
#     lines1 = open(io.StringIO(uploaded['dev.en'].decode('utf-8')),encoding='utf-8').read().strip().split('\n')
#     lines2 = open(io.StringIO(uploaded['dev.hi'].decode('utf-8')),encoding='utf-8').read().strip().split('\n')
    
    lines1 = open('en_train',encoding='utf-8').read().strip().split('\n')
    lines2 = open('hi_train',encoding='utf-8').read().strip().split('\n')
  
    print("len lines1 ",len(lines1))
    print("len lines2 ",len(lines2))
    print("line1 example")
    print(lines1[0])
    print("line2 example")
    print(lines2[0])
    
    if(reverse == True):
        for i in range(0,len(lines1)):
            tempLines = lines1[i].split(' ')
            temp =  Reverse(tempLines)
#             print("Temp")
#             print(temp)
            listToStr = ' '.join([str(elem) for elem in temp]) 
#             print(listToStr)
#             print(type(lines1[i]))
#             print(type(lines2[i]))
            
    print("line1 example AGAaIN")
    print(lines1[0])
    lines = []
    for i in range(0,len(lines1)):
        lines.append(str(lines1[i])+' \t '+str(lines2[i]))

    print("len(lines) ",len(lines))
    print("lines[0] ",str(lines[0]))
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    pairs = [[s for s in l.split('\t')] for l in lines]
    print("len(pairs) ",len(pairs))
    print(pairs[0])
    
    if reverse_lang:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [63]:
MAX_LENGTH =20
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [77]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('en', 'hi', True)
print("random.choice(pairs)")
print(random.choice(pairs))
orig_pairs = []
for p in pairs:
    temp = str(p[0]).split(' ')
    temp = Reverse(temp)
    listToStr = ' '.join([str(elem) for elem in temp]) 
#     print("listToStr")
#     print(listToStr)
    listToStr = str(listToStr)
    orig_pairs.append(listToStr+'\t'+str(p[1]))
#     print(orig_pairs[-1])
print("orig_pairs")
print(orig_pairs[0])
print(random.choice(orig_pairs))



Reading lines...
len lines1  200
len lines2  200
line1 example
but you will also be safe from eye diseases .
line2 example
बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।
line1 example AGAaIN
but you will also be safe from eye diseases .
len(lines)  200
lines[0]  but you will also be safe from eye diseases . 	 बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।
len(pairs)  200
['but you will also be safe from eye diseases . ', ' बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।']
Read 200 sentence pairs
Trimmed to 133 sentence pairs
Counting words...
Counted words:
en 622
hi 655
random.choice(pairs)
['They destroy the homocysteine which harms the nerves . ', ' ये नसों को क्षति पहुँचाने वाले होमोसिसटीन को नष्ट करते हैं ।']
orig_pairs
 . diseases eye from safe be also will you but	 बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।
 . hospitals maintained well in eye-surgeon by operated are cataract of patient the that After	 तत्पश्‍चात मोतियाबिंद वाले मरीजों का सुसज्जित अस्पतालों में नेत्रशल्यक द्वारा ऑपरेशन किया जाता है ।
